In [308]:
# coding=utf-8
"""
Prepare a Sentinel 2 L2A scene prepared with dc_preproc scripts for datacube indexing and ingestion.

This script uses the MTD_MSIL1C.xml or S2A_USER_MTD_L2A_DS*.xml files.

Created by Bruno Chatenoux (GRID-Geneva, the 22.11.2017)

Modified by John Roberts (CLCR, Leicetser, 14.08.2018)

To be run from datacube_env
"""
import click
import logging
from pathlib import Path
import re
import os
from dateutil import parser
import uuid
import rasterio.warp
from osgeo import osr
import yaml
import glob
import rasterio


def get_coords(geo_ref_points, spatial_ref):
    spatial_ref = osr.SpatialReference(spatial_ref)
    t = osr.CoordinateTransformation(spatial_ref, spatial_ref.CloneGeogCS())

    def transform(p):
        lon, lat, z = t.TransformPoint(p['x'], p['y'])
        return {'lon': lon, 'lat': lat}
    return {key: transform(p) for key, p in geo_ref_points.items()}

def populate_coord(doc):
    proj = doc['grid_spatial']['projection']
    doc['extent']['coord'] = get_coords(proj['geo_ref_points'], proj['spatial_reference'])

def get_projection(path):
    with rasterio.open(str(path)) as img:
        left, bottom, right, top = img.bounds
        return {
            'spatial_reference': str(str(getattr(img, 'crs_wkt', None) or img.crs.wkt)),
            'geo_ref_points': {
                'ul': {'x': left, 'y': top},
                'ur': {'x': right, 'y': top},
                'll': {'x': left, 'y': bottom},
                'lr': {'x': right, 'y': bottom},
            }
        }

def prep_dataset(fields, path):
    images_list = []
    image_glob = os.path.join(str(path), "GRANULE", '*', "IMG_DATA", "R*", "*.tif")
    images_list =glob.glob(image_glob)
    
    meta_glob = os.path.join(str(path), "DATASTRIP", "*", "MTD_DS.xml")
    metafile = glob.glob(meta_glob)[0]
    
    with open(metafile) as searchfile:
        for line in searchfile:
            if "DATASTRIP_SENSING_START" in line:
                aos = parser.parse(re.findall(r'\>(.*)\<', line)[0])
            if "DATASTRIP_SENSING_STOP" in line:
                los = parser.parse(re.findall(r'\>(.*)\<', line)[0])
            if "PROCESSING_CENTER" in line:
                name = re.findall(r'\>(.*)\<', line)[0]
            if "UTC_DATE_TIME" in line:
                creation_dt = parser.parse(re.findall(r'\>(.*)\<', line)[0])

    center_dt = aos + (los-aos)/2

    images = {im_path.replace('L2A_','') for im_path in images_list}
    
    #Get the actual projection of each image
    projdict = {image : get_projection(image_path) for image, image_path in zip(images, images_list)}
    print projdict
    doc = {
        'id': str(uuid.uuid4()),
        'processing_level': fields["level"],
        'product_type': fields["type"],
        'creation_dt':  str(creation_dt.strftime('%Y-%m-%d %H:%M:%S')),
        'platform': {'code': 'SENTINEL_2'},
        'instrument': {'name': fields["instrument"]},
        'acquisition': {
            'groundstation': {
                'name': name.replace('_', ''),
                'aos': str(aos.strftime('%Y-%m-%d %H:%M:%S')),
                'los': str(los.strftime('%Y-%m-%d %H:%M:%S'))
            }
        },
        'extent': {
            'from_dt': str(aos.strftime('%Y-%m-%d %H:%M:%S')),
            'to_dt': str(los.strftime('%Y-%m-%d %H:%M:%S')),
            'center_dt': str(center_dt.strftime('%Y-%m-%d %H:%M:%S'))
        },
        'format': {'name': 'Geotiff'},
        'grid_spatial': {
            'projection': projdict   
        },
        'image': {
            'tile': fields["tile"],
            'bands': images
        },
       
        'lineage': {'source_datasets': {}}
    }
    
    populate_coord(doc)
    return doc

def prepare_datasets(nbar_path):
    # Old; S2A_MSIL1C_20160717T104026_N0204_R008_T32TLS_20160718T060439
    # New; S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE
    match = re.match((r"(?P<sensor>S2A|S2B)_MSIL2A_"
                       r"(?P<productyear>[0-9]{4})"
                       r"(?P<productmonth>[0-9]{2})"
                       r"(?P<productday>[0-9]{2})T"
                       r"(?P<producthour>[0-9]{2})"
                       r"(?P<productminute>[0-9]{2})"
                       r"(?P<productsecond>[0-9]{2})_N"
                       r"(?P<baselinenumber>[0-9]{4})_R"
                       r"(?P<relativeorbit>[0-9]{3})_T"
                       r"(?P<tile>[A-Z0-9]{5})_"), nbar_path.stem)
    fields = match.groupdict()

    fields.update({'level': 'L2A',
        'type': 'dc_preproc',
        'instrument': 'MSI',
        'creation_dt': '%s-%s-%s 00:00:00' % (fields['productyear'], fields['productmonth'], fields['productday'])})

    nbar = prep_dataset(fields, nbar_path)
    return (nbar, nbar_path)




In [309]:

def test_main(datasets):
    """Takes a list of SAFE files and produces a yaml for each"""
    logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s', level=logging.ERROR)
    print("Entering program")
    for dataset in datasets:
        path = Path(dataset)
        print("Processing", path)
        logging.info("Processing %s", path)
        documents = prepare_datasets(path)
        print("completed preparing dataset, about to write output")
        dataset, folder = documents
        yaml_path = str(folder.joinpath('agdc-metadata.yaml'))
        logging.info("Writing %s", yaml_path)
        with open(yaml_path, 'w') as stream:
            yaml.dump(dataset, stream)



In [310]:
safe_file = r"/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE"
test_main([safe_file])

Entering program
('Processing', PosixPath('/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE'))
{'/home/cubo/john_r_s2_ingestion/S2A_MSI20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/T18NWG_A011462_20170901T153206/IMG_DATA/R20m/T18NWG_20170901T152641_WVP_20m.tif': {'geo_ref_points': {'ul': {'y': 200010.0, 'x': 500010.0}, 'll': {'y': 90210.0, 'x': 500010.0}, 'lr': {'y': 90210.0, 'x': 609810.0}, 'ur': {'y': 200010.0, 'x': 609810.0}}, 'spatial_reference': 'PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_north

KeyError: 'geo_ref_points'

In [187]:
"""IDEAM datacube preprocessing script. Includes:
jp2 conversion to geotiff using old GDAL
Duplicate product cleaner
By John Roberts, jfr10@le.ac.uk"""

from collections import deque
from xml.etree import ElementTree
import subprocess
import gdal, osr
import re

gdal.UseExceptions()

def extract_metadata(jp2_path):
    """Extracts the XML metadata in a jp2 geo-image as a bytestring. No GDAL."""
    with open(jp2_path, "rb") as jp2_bytes:
        # This iterates through the jp2 file byte-by-byte, looking 
        # for the start tag FeatureCollection. Once found, it reads
        # bytes into output string until the matching closing tag
        # is found. Finally, it strips out extra characters
        testqueue = deque(maxlen=24)
        testqueue.append(jp2_bytes.read(1))
        out = b""
        # Look for first xml element
        while b"<gml:FeatureCollection x" not in b"".join(testqueue):
            testqueue.append(jp2_bytes.read(1))
        out += b"".join(testqueue)
        # Look for the bytes that explicitly are NOT xml
        while b'\x00\x00\x00\x00' not in b"".join(testqueue):
            testqueue.append(jp2_bytes.read(1))
            out += testqueue[-1]
        # Removing newline characters and stripping extra bytes at end
        # out = out.replace("\n", "")
        out = out[:out.rfind(b">")+1]
    return out


def get_geotransform_from_metadata(metadata_string):
    """Returns the gdal geotransform, raster size and CRS string from metadata."""
    metadata = ElementTree.fromstring(metadata_string)
    namespaces = {"ns0":r"http://www.opengis.net/gml"}

    origin_point = metadata.find(r".//ns0:origin/ns0:Point", namespaces)
    projection = origin_point.attrib.get("srsName")

    origin_pos = origin_point.find(r".//ns0:pos", namespaces)
    origin_pos_tuple = origin_pos.text.split(" ")
    origin_pos_tuple = tuple(int(coord) for coord in origin_pos_tuple)

    size_node = metadata.find(r".//ns0:GridEnvelope/ns0:high", namespaces)
    size_tuple = size_node.text.split(" ")
    size_tuple = tuple(int(coord) for coord in size_tuple)

    res_nodes = metadata.findall(r".//ns0:RectifiedGrid/ns0:offsetVector", namespaces)
    x_res = int(res_nodes[0].text.split(" ")[0])
    y_res = int(res_nodes[1].text.split(" ")[1])

    # build the geotransform
    gt = (origin_pos_tuple[0], x_res, 0, origin_pos_tuple[1], 0, y_res)
    return gt, size_tuple, projection


def add_projection_and_extent_to_tiff(tif_path, metadata_tuple):
    """Restores geospatial data to a tiff. Takes output from
    get_geotransform_from_metadata"""
    gt, size, projection = metadata_tuple
    EPSG = int(projection.split("::")[-1])
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(EPSG)
    image = gdal.Open(tif_path, gdal.GA_Update)
    image.SetGeoTransform(gt)
    image.SetProjection(srs.ExportToWkt())
    print image.GetProjection()
    image = None
    

def convert_jp2_to_tiff(jp2_path, tif_path):
    """Converts a jp2 to a tif (NOT Geotiff) using Jpeg2000 tools"""
    subprocess.call(["opj_decompress", "-i", jp2_path, "-o", tif_path])
    return tif_path


def replace_jp2_with_geotiff(jp2_path, remove_old=False):
    """Will replace a jp2 with a geotiff of the same name.
    Can remove old jp2"""
    tif_path = jp2_path[-4]+".tif"
    metadata_string = extract_metadata(jp2_path)
    metadata = get_geotransform_from_metadata(metadata_string)
    convert_jp2_to_tiff(jp2_path, tif_path)
    add_projection_and_extent_to_tiff(tif_path, metadata)
    if remove_old:
        os.rm(jp2_path)

        
def convert_dir_to_geotiff(jp2_dir, remove_old=False):
    """Converts an entire directory of jp2 to geotiff"""
    images = glob.glob(os.path.join(jp2_dir, "*.jp2"))
    for image in images:
        replace_jp2_with_geotiff(image, remove_old)
        
    
def remove_extra_s2_images(safe_file):
    """Removes images stored at a higher resolution than required"""
    # Two regexes that match every band except those listed after the ?!
    re_20_m=r"L2A_\S+_B(?!05|06|07|8A|11|12)\S+(.jp2|.tif)"
    re_60_m=r"L2A_\S+_B(?!01|09|10)\S+(.jp2|.tif)"
    
    base_glob = os.path.join(safe_file, "GRANULE", '*', "IMG_DATA")
    glob_20_m = os.path.join(base_glob, "R20m")
    glob_60_m = os.path.join(base_glob, "R60m")
    path_20_m = glob.glob(glob_20_m)[0]
    path_60_m = glob.glob(glob_60_m)[0]
    list_20_m = os.listdir(path_20_m)
    list_60_m = os.listdir(path_60_m)

    
    regex_output_20 = list(re.match(re_20_m, path) for path in list_20_m)
    to_delete_20 = list(os.path.join(path_20_m,match.group(0))
                        for match in regex_output_20 if match)
    
    regex_output_60 = list(re.match(re_60_m, path) for path in list_60_m)
    to_delete_60 = list(os.path.join(path_60_m,match.group(0))
                        for match in regex_output_60 if match)
    
    for tbd in to_delete_20+to_delete_60:
        print "Deleting "+tbd+"\n"
        os.remove(tbd)
    
    


In [189]:
test_image = r'/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/L2A_T18NWG_20170901T152641_B05_20m.jp2'
test_out = r'/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/L2A_T18NWG_20170901T152641_B05_20m.tif'
jp2_dir = (r"/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/")
convert_dir_to_geotiff(jp2_dir)

['/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/L2A_T18NWG_20170901T152641_TCI_20m.jp2', '/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/L2A_T18NWG_20170901T152641_B12_20m.jp2', '/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/L2A_T18NWG_20170901T152641_SCL_20m.jp2', '/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/L2A_T18NWG_20170901T152641_AOT_20m.jp2', '/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R20m/L2A_T18NWG_20170901T152641_WVP_20m.jp2', '/home/cubo/john_r_

PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32618"]]
PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["

In [274]:
safe_file = "/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE"
remove_extra_s2_images(safe_file)

In [16]:
foo = Path(test_fp).stem

In [19]:
match = re.match((r"(?P<sensor>S2A|S2B)_MSIL2A_"
                       r"(?P<productyear>[0-9]{4})"
                       r"(?P<productmonth>[0-9]{2})"
                       r"(?P<productday>[0-9]{2})T"
                       r"(?P<producthour>[0-9]{2})"
                       r"(?P<productminute>[0-9]{2})"
                       r"(?P<productsecond>[0-9]{2})_N"
                       r"(?P<baselinenumber>[0-9]{4})_R"
                       r"(?P<relativeorbit>[0-9]{3})_T"
                       r"(?P<tile>[A-Z0-9]{5})_"), foo)

In [28]:
match.groupdict()

{'baselinenumber': '0205',
 'productday': '01',
 'producthour': '15',
 'productminute': '26',
 'productmonth': '09',
 'productsecond': '41',
 'productyear': '2017',
 'relativeorbit': '025',
 'sensor': 'S2A',
 'tile': '18NWG'}

In [30]:
import glob

In [286]:
    images_list = []
    
    image_glob = os.path.join(safe_file, "GRANULE", '*', "IMG_DATA", "R*", "*tif")
    
    images_list =glob.glob(image_glob)

In [287]:
  {im_path.stem.replace('L2A_',''): {
        'path': str(im_path.relative_to(path))   
    } for im_path in foo.glob('*.jp2')}

NameError: name 'foo' is not defined

In [288]:
images_list[0]

'/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R60m/L2A_T18NWG_20170901T152641_TCI_60m.tif'

In [304]:
import gdal
image = gdal.Open(images_list[0])
proj = image.GetProjection()
print proj

In [290]:
test_proj = get_projection(images_list[0])
test_doc = {
    "grid_spatial": {
        "projection": test_proj
    },
    "extent":{}
}
populate_coord(test_doc)
test_doc

NotImplementedError: Wrong number or type of arguments for overloaded function 'CoordinateTransformation_TransformPoint'.
  Possible C/C++ prototypes are:
    OSRCoordinateTransformationShadow::TransformPoint(double [3])
    OSRCoordinateTransformationShadow::TransformPoint(double [3],double,double,double)


In [71]:
baz = gdal.Open(images_list[6])

2018-08-14 14:45:11,212 ERROR CPLE_OpenFailed in `/home/cubo/john_r_s2_ingestion/S2A_MSIL2A_20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/L2A_T18NWG_A011462_20170901T153206/IMG_DATA/R60m/L2A_T18NWG_20170901T152641_B07_60m.jp2' not recognized as a supported file format.



In [72]:
gdal.Info()

TypeError: Info() takes exactly 1 argument (0 given)

In [73]:
gdal.__version__

AttributeError: 'module' object has no attribute '__version__'

In [76]:
osgeo.__version__

'2.1.0'

In [75]:
import osgeo

In [99]:
images = {im_path.replace('L2A_','') for im_path in images_list}
projdict = {image : get_projection(image_path) for image, image_path in zip(images, images_list)}

In [100]:
projdict

{'/home/cubo/john_r_s2_ingestion/S2A_MSI20170901T152641_N0205_R025_T18NWG_20170901T153206.SAFE/GRANULE/T18NWG_A011462_20170901T153206/IMG_DATA/R60m/T18NWG_20170901T152641_B01_60m.tif': {'geo_ref_points': {'ll': {'x': 499980.0,
    'y': 90240.0},
   'lr': {'x': 609780.0, 'y': 90240.0},
   'ul': {'x': 499980.0, 'y': 200040.0},
   'ur': {'x': 609780.0, 'y': 200040.0}},
  'spatial_reference': 'PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32618"]]'}}